In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\Olívia\Desktop\batatinhas\coleta_cebrap
Caminho no path.


In [2]:
import geopandas as gpd
import os

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel,
)

# Ocorrências de alagamentos por distrito
Com base nas intersecções que temos, podemos somar os pontos para sabermos o total em cada distrito

In [3]:
data_path = get_data_diretorio()
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências
Este notebook é dependente dos arquivos resultantes dos notebooks "overlay_pontos_inundacao_2024" e "../../arborizacao_viaria/malha_distritos"

In [4]:
gdf_distritos = gpd.read_parquet(
    os.path.join(
        assets_path,
        'distrito_ibge.parquet'
    )
)

In [5]:
overlay_alaga = gpd.read_parquet(
    os.path.join(
        assets_path,
        'areas_alagamento',
        'interseccao_alagamentos2024_distritos.parquet'
    )
)

# Soma de pontos por distrito

In [6]:
for i, distrito in enumerate(gdf_distritos['CD_DIST']):
    (
        gdf_distritos
        .loc[gdf_distritos['CD_DIST']==distrito,
        'qt_alaga_ocor']
    )= len(
            overlay_alaga
            .loc[overlay_alaga['CD_DIST'] == distrito]
        )

In [7]:
gdf_distritos.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_alaga_ocor
25,3550308,São Paulo,355030826,CONSOLACAO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,3.786948,53144,35777,"POLYGON ((331116.481 7395552.671, 331137.408 7...",0.0
19,3550308,São Paulo,355030820,CARRAO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,7.785398,84397,37987,"POLYGON ((342726.019 7393623.533, 342704.551 7...",6.0


# Conferir

Distritos sem pontos de ocorrências de inundações:

In [8]:
zero_dists=(
    gdf_distritos
    .loc[gdf_distritos['qt_alaga_ocor']==float(0)]
)
zero_dists.shape

(41, 15)

In [9]:
zero_dists.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_alaga_ocor
38,3550308,São Paulo,355030839,JACANA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,7.473932,87329,34705,"POLYGON ((339057.719 7403206.175, 339043.253 7...",0.0
50,3550308,São Paulo,355030851,MANDAQUI,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,13.202709,103665,45143,"POLYGON ((332902.165 7405221.112, 332904.48 74...",0.0


Distrito com o maior número de ocorrências de inundações:

In [10]:
(
    gdf_distritos
    .loc[gdf_distritos['qt_alaga_ocor']==gdf_distritos['qt_alaga_ocor'].max()]
)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_alaga_ocor
43,3550308,São Paulo,355030844,JARDIM HELENA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,9.133494,129409,48496,"POLYGON ((355699.653 7401143.506, 355536.385 7...",36.0


Distrito com o menor número de alagamentos (excluindo os sem ocorrência nenhuma):

In [11]:
min_positiv=(
    gdf_distritos
    .loc[gdf_distritos['qt_alaga_ocor']>0]
)
min_dists=(
    gdf_distritos
    .loc[gdf_distritos['qt_alaga_ocor']==min_positiv['qt_alaga_ocor'].min()]
)
min_dists.shape

(24, 15)

In [12]:
min_dists.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_alaga_ocor
28,3550308,São Paulo,355030829,FREGUESIA DO O,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,11.038192,137240,59796,"POLYGON ((327061.163 7399198.253, 327009.96 73...",1.0
91,3550308,São Paulo,355030892,VILA MEDEIROS,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,7.875923,114939,48273,"POLYGON ((338883.438 7400767.46, 338872.694 74...",1.0


Nós vimos que a subprefeitura com mais pontos de ocorrências de inundação é a de São Miguel Paulista. Vamos confirmar se mantém quando calculamos pelos distritos:

In [13]:
overlay_alaga.columns

Index(['data', 'ocorrencia', 'nm_subpref', 'id_ocorrencias', 'CD_MUN',
       'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_RGINT', 'NM_RGINT', 'CD_RGI',
       'NM_RGI', 'CD_CONCURB', 'NM_CONCURB', 'AREA_KM2', 'total_pop',
       'total_dom', 'geometry'],
      dtype='object')

In [14]:
subpref_confere = gdf_distritos.copy()
for i, distrito in enumerate(subpref_confere['CD_DIST']):
    if overlay_alaga['CD_DIST'].str.contains(distrito).any():
        (
            subpref_confere
            .loc[subpref_confere['CD_DIST']==distrito, 
            'subpref']
        )=(
            overlay_alaga
            .loc[overlay_alaga['CD_DIST']==distrito]
            .head(1)['nm_subpref']
            .values[0]
        )

In [15]:
subpref_confere.sample(3)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_alaga_ocor,subpref
53,3550308,São Paulo,355030854,MORUMBI,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,11.462372,43690,20710,"POLYGON ((325192.046 7387331.143, 325166.217 7...",0.0,NaN
39,3550308,São Paulo,355030840,JAGUARA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,4.533105,24730,10783,"POLYGON ((320979.577 7398155.785, 320973.947 7...",0.0,NaN
14,3550308,São Paulo,355030815,CAMPO BELO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,8.842744,71034,36663,"POLYGON ((329439.691 7384123.427, 329399.519 7...",3.0,SA - SANTO AMARO


# Salvar GDF

In [16]:
save_parquet_excel(
    gdf_distritos,
    fname='qt_ocorrencias_alaga_distrito',
    data_path = assets_path,
    data_subpath= 'areas_alagamento'
)